In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

chat_model = ChatNVIDIA(
  model="meta/llama-3.1-8b-instruct",
  api_key=os.environ["NVIDIA_API_KEY"], 
  temperature=0.0,
)

In [3]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computer_science_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [4]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computer_science_template
    }
]

In [5]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utils.math import cosine_similarity

In [6]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = prompt | chat_model
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [7]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

Return a single candidate prompt that you think suite the best, \
to provide more specific context to the raw prompt \
without changing the base context from original prompt.\

Raw prompt: {prompt}
Candidate prompt: `your_prompt` + `raw prompt`

REMEMBER: Only return the return the single prompt and not anything else
"""

In [8]:
multi_prompt_template = PromptTemplate.from_template(MULTI_PROMPT_ROUTER_TEMPLATE)

In [9]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = default_prompt | chat_model

In [10]:
embeddings = NVIDIAEmbeddings()
prompt_templates = [physics_template, math_template, history_template, computer_science_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

In [11]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["input"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)
    most_similar = prompt_templates[similarity.argmax()]
    info = prompt_infos[similarity.argmax()]
    print(f"{info['name']}: {info['description']}")
    print("\n====================================\n")
    
    return PromptTemplate.from_template(most_similar)


router_chain =  multi_prompt_template | chat_model | StrOutputParser() | {"input": RunnablePassthrough()} | RunnableLambda(prompt_router) | chat_model | StrOutputParser()

In [12]:
print(router_chain.invoke('what is 2 + 2'))

math: Good for answering math questions


A simple question to start with!

To break down the problem, I'll identify the two components:

1. The first number to add: 2
2. The second number to add: 2

Now, I'll perform the arithmetic operation:

1. Add the two numbers: 2 + 2
2. Combine the numbers: 2 + 2 = 4

Therefore, the answer to the question is: 4


In [13]:
print(router_chain.invoke('Why earth have gravity?'))

physics: Good for answering questions about physics


The Earth's gravitational force is a fascinating topic.

The Earth's gravitational force is a result of its mass and the way it warps the fabric of spacetime around it. According to Einstein's theory of General Relativity, massive objects like the Earth create a curvature in spacetime, which we experience as gravity.

Think of it like this: imagine spacetime as a trampoline. If you place a heavy object, like a bowling ball, on the trampoline, it will warp and curve, creating a dent. Now, if you roll a marble nearby, it will follow the curvature of the trampoline and move towards the bowling ball. That's essentially what's happening with the Earth's gravity – the Earth's mass is warping spacetime, and objects with mass, like us, are following that curvature.

The strength of the gravitational force depends on the mass of the object and the distance from the object. That's why the Earth's gravity is stronger at its surface than it is 

In [14]:
print(router_chain.invoke('What is AI?'))

computer science: Good for answering computer science questions


I'd be delighted to explain the concept of Artificial Intelligence in simple terms.

**What is Artificial Intelligence (AI)?**

Artificial Intelligence is a branch of computer science that focuses on creating intelligent machines that can think and act like humans. In simpler terms, AI is a way to make computers and machines "smart" so they can perform tasks that typically require human intelligence, such as:

1. Learning from experience
2. Solving complex problems
3. Recognizing patterns
4. Making decisions
5. Interacting with humans in a natural way

Think of AI like a super-smart robot that can:

* Understand natural language (like you and me)
* See and recognize images, objects, and patterns
* Learn from data and improve over time
* Make predictions and decisions based on that data
* Interact with humans in a way that feels natural and intuitive

**How does AI work?**

AI works by using algorithms, which are like rec